# Data Manipulator

Used to manipulate data and store them as intermediate result.

## Forward Return

Caculate the forward return as label for latter use

### Close to Close Return

This type of return is not robust, and may serverely effect the model performance, use with caution

In [ ]:
import pandas as pd

data = pd.read_parquet('./kline_daily/', columns=['close', 'adjfactor'])
close = data['close'] * data['adjfactor']
ret = (close.groupby(level=1).shift(-2) / close.groupby(level=1).shift(-1) - 1).dropna()
ret = ret.loc[~ret.index.duplicated(keep='first')]
nret = ret.groupby(level=0).apply(lambda x: (x - x.mean()) / x.std())
ret.to_frame(name='label').to_parquet('./intermediate/forward_return_1d_close_close.parquet')
nret.to_frame(name='label').to_parquet('./intermediate/forward_return_1d_close_close_normalized.parquet')

### VWAP Return

This type of return is more plausible, and more like the true market environment

In [1]:
import pandas as pd

data = pd.read_parquet('./derivative_indicators/vwap.parquet')
vret = data.groupby(level=1).shift(-2) / data.groupby(level=1).shift(-1) - 1
vret = vret.dropna()
vret.to_parquet('../data/intermediate/forward_return/1d_vwap_vwap.parquet')
nvret = vret.groupby(level=0).apply(lambda x: (x - x.mean()) / x.std())
nvret.to_parquet('../data/intermediate/forward_return/1d_vwap_vwap_normalized.parquet')